# AmericaSave Mortgage Assessment

## Project

Your team must develop a real-time SQL assignment algorithm that automatically matches prospective customers with the best travel agent available. Agents not only guide customers through the booking process but also upsell luxury packages, exclusive excursions, and custom accommodations. At the end of each journey, customers rate their experience with their travel agent. Your solution should receive details about a customer (listed below) and return a stack-ranked list of travel agents ordered from best to worst.

## Requirements

- Provide a written overview of your model and the approach you chose
- Provide SQL Code that can be executed without errors. If your model requires the building of new tables, stored procedures or functions, make sure you provide the SQL code that creates them

## Overview

The goal of this workflow was to build an intelligent agent assignment system that balances workload fairly, adapted dynamically to booking activity, and ensured customers were matched with the most qualified agents. By integrating historical booking data and agent performance metrics, the system kept track of each agent’s availability and continuously updated rankings based on their activity. Automated triggers manage incoming customer assignments and respond to booking status changes in real time, enabling a responsive and scalable approach to agent deployment.

## Step-by-step instruction

- <u>Step 1</u>: Merged the `bookings` and `assignment_history` tables to incorporate the `AgentID` field into the `bookings` table. The resulting table was named `bookings_2`, which would be used going forward.
- <u>Step 2</u>: Added `load` and `agent_rank` columns to the `space_travel_agents` table.
    - The `load` column represented the number of active customer assignments for each agent.
    - The `agent_rank` column prioritized agents based on minimal `load`, highest `YearsOfService`, and best `AverageCustomerServiceRating`.
    - These metrics would be updated regularly to ensure incoming customers were matched with the most suitable agents.
- <u>Step 3</u>: Initialized the `load` and `agent_rank columns`.
    - All agents were initially assigned a `load` value of 0.
    - `agent_rank` was computed in the following order of precedence: `load` in ascending order, `YearsOfService` in descending order, and `AverageCustomerServiceRating` in descending order.
- <u>Step 4</u>: Created a new table, `new_customer`, containing the following fields: `CustomerName`, `CommunicationMethod`, `LeadSource`, `Destination`, and `LaunchLocation`.
- <u>Step 5</u>: Defined the following triggers to automate assignment workflows and load tracking:
    - `updating_assignment_history`: Inserted a record into `assignment_history` with the fields `CustomerName`, `AssignmentID`, `CommunicationMethod`, `LeadSource`, `AssignedDateTime`, and `AgentID`. Note: `AssignedDateTime` was set to the current time plus 56 years.
    - `updating_bookings_2`: Inserted data into `bookings_2`, including `BookingID`, `AssignmentID`, `Destination`, `LaunchLocation`, `BookingStatus`, and `AgentID`.
    - `updating_loads`: Incremented the `load` value in `space_travel_agents` whenever a new customer was inserted into `new_customer`.
    - `update_loads_subtracting`: Decremented the agent’s `load` in `space_travel_agents` if a `booking`’s `BookingStatus` changed from 'Pending' to either 'Confirmed' or 'Cancelled'.
    - `recompute_agent_rank_on_load`: Recalculated `agent_rank` in response to changes in `load`. Since `agent_rank` was dynamic, this trigger ensured that it remained up to date as assignments shift.
- <u>Step 6</u>: To evaluate the algorithm’s behavior, I tested two scenarios:
    – <u>Scenario 1</u>: The `new_customer` table was updated to simulate the addition of new users.
    – <u>Scenario 2</u>: The `BookingStatus` field in `bookings_2` was modified to reflect status changes.

## Import and connect to the database

In [1]:
import sqlite3

In [2]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

In [3]:
%load_ext sql
%sql sqlite:///my_database.db

Connecting to 'sqlite:///my_database.db'

In [4]:
# Delete all tables from the database. This is useful when rerunning the notebook.

In [5]:
# Get all user-defined tables (excluding internal SQLite tables)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
tables = cursor.fetchall()

# Drop each table
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]};")

In [6]:
# Read the tables
table_list = ['assignment_history SQL Table.txt', 'bookings SQL Table.txt', 'space_travel_agents SQL Table.txt']

for t in table_list:
    with open(t, 'r') as f:
        script = f.read()
    cursor.executescript(script)

## Step 1: Merge tables
I merged tables `bookings` and `assignment_history` so that `bookings` included `AgentID` information. I named this table `bookings_2` and and I would be using it going forward.

In [7]:
%%sql

CREATE TABLE bookings_2 AS
SELECT B.*,
        AH.AgentID
FROM bookings AS B
LEFT JOIN assignment_history AS AH USING(AssignmentID)

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 2: Added `load` and `agent_rank` columns

I created a `load` column with a default value of 0 to track how many active customer assignments each agent had. Whenever an agent was assigned to a new customer, their corresponding `load` value was incremented to reflect their current workload. Conversely, after completing a customer's request, the `load` was decremented by 1.

In [8]:
%%sql 

ALTER TABLE space_travel_agents
ADD COLUMN load INTEGER DEFAULT 0;

ALTER TABLE space_travel_agents
ADD COLUMN agent_rank INTEGER;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 3: Initialized the `load` and `agent_rank columns`

As an initial setup before implementing triggers, I manually iterated through the `bookings_2` table and incremented the `load` value for each agent whenever a booking record had a `BookingStatus` of 'Pending'. This approach ensured that the agent workloads accurately reflected all active customer assignments prior to automating the process with triggers.

In [9]:
%%sql

UPDATE space_travel_agents
SET load = load + (
  SELECT COUNT(*)
  FROM bookings_2
  WHERE bookings_2.AgentID = space_travel_agents.AgentID
    AND bookings_2.BookingStatus = 'Pending'
)

Running query in 'sqlite:///my_database.db'

30 rows affected.

++
||
++
++

In [10]:
%%sql

UPDATE space_travel_agents
SET agent_rank = (
  SELECT COUNT(*)
  FROM space_travel_agents AS b
  WHERE b.load < space_travel_agents.load
     OR (b.load = space_travel_agents.load AND b.YearsOfService > space_travel_agents.YearsOfService)
     OR (b.load = space_travel_agents.load AND b.YearsOfService = space_travel_agents.YearsOfService AND b.AverageCustomerServiceRating > space_travel_agents.AverageCustomerServiceRating)
) + 1;

Running query in 'sqlite:///my_database.db'

30 rows affected.

++
||
++
++

In [11]:
%sql SELECT * FROM space_travel_agents LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,FirstName,LastName,Email,JobTitle,DepartmentName,ManagerName,SpaceLicenseNumber,YearsOfService,AverageCustomerServiceRating,load,agent_rank
1,Aurora,Tanaka,aurora.tanaka@astra2081.com,Senior Space Travel Agent,Interplanetary Sales,Lyra Chen,SL-2081-001,12,4.0,0,5
2,Kai,Rodriguez,kai.rodriguez@astra2081.com,Space Travel Agent,Luxury Voyages,Lyra Chen,SL-2081-002,7,4.0,1,21
3,Nova,Singh,nova.singh@astra2081.com,Senior Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-003,15,4.0,0,2
4,Leo,Kim,leo.kim@astra2081.com,Space Travel Agent,Interplanetary Sales,Lyra Chen,SL-2081-004,5,4.0,0,10
5,Vera,Nguyen,vera.nguyen@astra2081.com,Space Travel Agent,Luxury Voyages,Zane Holloway,SL-2081-005,3,3.9,1,24


### Create agent_rank_tracker

In [12]:
%%sql

CREATE TABLE IF NOT EXISTS agent_rank_tracker (
    AgentID INTEGER PRIMARY KEY,
    agent_rank INTEGER 
);

INSERT INTO agent_rank_tracker (AgentID, agent_rank)
    SELECT a.AgentID,
        (
            SELECT COUNT(*)
            FROM space_travel_agents AS b
            WHERE b.load < a.load
              OR (b.load = a.load AND b.YearsOfService > a.YearsOfService)
              OR (b.load = a.load AND b.YearsOfService = a.YearsOfService AND b.AverageCustomerServiceRating > a.AverageCustomerServiceRating)
         ) + 1
    FROM space_travel_agents AS a;

Running query in 'sqlite:///my_database.db'

30 rows affected.

++
||
++
++

In [13]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
6,1
3,2
27,2
24,4
1,5


## Step 4: Created `new_customer` table

In [14]:
%%sql

CREATE TABLE new_customer (
    CustomerName VARCHAR(100) PRIMARY KEY,
    CommunicationMethod VARCHAR(50) NOT NULL CHECK (CommunicationMethod IN ('Phone Call', 'Text')),
    LeadSource VARCHAR(50) NOT NULL CHECK (LeadSource IN ('Organic', 'Bought')),
    Destination VARCHAR(50) NOT NULL CHECK (Destination IN ('Mars', 'Europa', 'Titan', 'Venus', 'Ganymede')),
    LaunchLocation VARCHAR(200) NOT NULL CHECK (LaunchLocation IN ('Dallas-Fort Worth Launch Complex', 
    'New York Orbital Gateway', 'Dubai Interplanetary Hub',
    'Tokyo Spaceport Terminal', 'London Ascension Platform', 'Sydney Stellar Port'))
    )

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 5: Defined triggers

### Updated assignment_history

In [15]:
%%sql

CREATE TRIGGER updating_assignment_history
AFTER INSERT ON new_customer
FOR EACH ROW
BEGIN

    -- Insert assignment_history
    INSERT INTO assignment_history (
                CustomerName,
                AssignmentID,
                CommunicationMethod,
                LeadSource,
                AssignedDateTime,
                AgentID
                )
    VALUES (
            NEW.CustomerName,
            (SELECT IFNULL(MAX(AssignmentID), 0) + 1 FROM assignment_history),
                    NEW.CommunicationMethod,
                    NEW.LeadSource,
                    datetime('now', '+56 years'),
            (SELECT AgentID FROM agent_rank_tracker ORDER BY agent_rank LIMIT 1)
    );
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

### Inserted into bookings_2

In [16]:
%%sql

CREATE TRIGGER updating_bookings_2
AFTER INSERT ON assignment_history
FOR EACH ROW
BEGIN
    
    INSERT INTO bookings_2 (
                BookingID,
                AssignmentID,
                Destination,
                LaunchLocation,
                BookingStatus,
                AgentID
                )
    VALUES (
            (SELECT IFNULL(MAX(BookingID), 0) + 1 FROM bookings_2),
                    NEW.AssignmentID,
                    (SELECT Destination FROM new_customer WHERE CustomerName = NEW.CustomerName),
                    (SELECT LaunchLocation FROM new_customer WHERE CustomerName = NEW.CustomerName),
                    'Pending',
                    NEW.AgentID
            );
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

### Updated load if new row was added to table `bookings_2`

In [17]:
%%sql

CREATE TRIGGER updating_loads
AFTER INSERT ON bookings_2
FOR EACH ROW
BEGIN
    UPDATE space_travel_agents
    SET load = load + 1
    WHERE AgentID = NEW.AgentID AND NEW.BookingStatus = 'Pending';
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

### Changed `BookingStatus` from "Pending" to "Cancelled" or "Confirmed"

In [18]:
%%sql

CREATE TRIGGER update_loads_subtracting
AFTER UPDATE OF BookingStatus ON bookings_2
FOR EACH ROW
    WHEN OLD.BookingStatus = 'Pending' AND NEW.BookingStatus IN ('Confirmed', 'Cancelled')
    BEGIN
        UPDATE space_travel_agents
        SET load = MAX(load - 1, 0)
        WHERE AgentID = NEW.AgentID;
    END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

### Recomputed agent_rank

In [19]:
%%sql
    
CREATE TRIGGER recompute_agent_rank_on_load
AFTER UPDATE OF load ON space_travel_agents
FOR EACH ROW
BEGIN
    DELETE FROM agent_rank_tracker;

    INSERT INTO agent_rank_tracker (AgentID, agent_rank)
    SELECT a.AgentID,
            (SELECT COUNT(*)
                FROM space_travel_agents AS b
                WHERE b.load < a.load
                OR (b.load = a.load AND b.YearsOfService > a.YearsOfService)
                OR (b.load = a.load AND b.YearsOfService = a.YearsOfService
                AND b.AverageCustomerServiceRating > a.AverageCustomerServiceRating)
            ) + 1
FROM space_travel_agents AS a;
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 6: Evaluated the algorithm

### Scenario 1: Inserted new rows to `new_customer`

Prior to inserting new records, the top five agents ranked were `AgentIDs` 6, 3, 27, 24, and 1.

In [20]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
6,1
3,2
27,2
24,4
1,5


I added two rows to the new_customer table. 
- Kate Nguyen, located in Houston, TX, contacted our company via text. Her friend had previously taken a trip with us, and she expressed interest in traveling to Mars. The closest launch location for her was the Dallas–Fort Worth Launch Complex.
- John Doe called us from Sydney after discovering our company through Facebook Ads. He was interested in a trip to Europa departing from the Sydney Stellar Port.

In [21]:
%%sql

INSERT INTO new_customer (
                        CustomerName, 
                        CommunicationMethod,
                        LeadSource, 
                        Destination, 
                        LaunchLocation
                        )
VALUES (
        'Kate Nguyen', 
        'Text', 
        'Organic', 
        'Mars', 
        'Dallas-Fort Worth Launch Complex'),
        (
        'Jonh Doe', 
        'Phone Call', 
        'Bought', 
        'Europa', 
        'Sydney Stellar Port'
        )

Running query in 'sqlite:///my_database.db'

2 rows affected.

++
||
++
++

The `assignment_history` table was updated correctly, including the increment of `AssignmentID`. Notably, the top two agents, `AgentID` 6 and `AgentID` 3, were correctly assigned to Kate and John, respectively. Additionally, the AssignedDateTime field accurately reflected the intended future timestamp.

In [22]:
%sql SELECT * FROM assignment_history ORDER BY AssignmentID DESC

Running query in 'sqlite:///my_database.db'

AssignmentID,AgentID,CustomerName,CommunicationMethod,LeadSource,AssignedDateTime
452,3,Jonh Doe,Phone Call,Bought,2081-07-10 23:16:20
451,6,Kate Nguyen,Text,Organic,2081-07-10 23:16:20
450,11,Mira Cruz,Phone Call,Bought,2081-04-10 15:00:00
449,20,Arlo King,Text,Organic,2081-04-10 13:50:00
448,2,Tinsley Ross,Text,Bought,2081-04-10 12:40:00
447,16,Elowen Bell,Phone Call,Organic,2081-04-10 11:30:00
446,24,Callahan Rivera,Text,Bought,2081-04-10 10:20:00
445,7,Kylan Scott,Text,Organic,2081-04-10 09:10:00
444,28,Zya Lewis,Phone Call,Bought,2081-04-09 14:55:00
443,13,Azalea Brooks,Text,Organic,2081-04-09 13:45:00


Reviewing the `agent_rank_tracker`, the `agent_rank` values were updated correctly. `AgentID` 27, which was previously ranked third before the `new_customer` table was updated, moved to first on the list.

In [23]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 10

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
27,1
24,2
1,3
16,3
19,5
11,6
25,7
4,8
23,9
29,10


### Scenario 2: Changed `BookingStatus` from "Pending" to "Cancelled"

I reselected AgentID 3, who was ranked 17 after accepting a new request. Upon removing the assigned load, AgentID 3 returned to the top rank. This scenario would serve as a valuable sanity check, confirming the accuracy of the ranking logic.

In [24]:
%sql SELECT * FROM agent_rank_tracker WHERE AgentID = 3

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
3,17


Changed the `BookingStatus` for `AgentID` 3 from 'Pending' to 'Cancelled'.

In [25]:
%%sql

SELECT * 
FROM bookings_2
WHERE AgentID = 3 AND BookingStatus = 'Pending'

Running query in 'sqlite:///my_database.db'

BookingID,AssignmentID,BookingCompleteDate,CancelledDate,Destination,Package,LaunchLocation,DestinationRevenue,PackageRevenue,TotalRevenue,BookingStatus,AgentID
414,452,None,None,Europa,None,Sydney Stellar Port,None,None,None,Pending,3


Let's change the BookingStatus of BookingID = 414 into 'Cancelled'.

In [26]:
%%sql

UPDATE bookings_2
SET BookingStatus = 'Cancelled'
WHERE AgentID = 3 AND BookingID = 414;

SELECT * 
FROM bookings_2
WHERE AgentID = 3 AND BookingID = 414;

Running query in 'sqlite:///my_database.db'

1 rows affected.

BookingID,AssignmentID,BookingCompleteDate,CancelledDate,Destination,Package,LaunchLocation,DestinationRevenue,PackageRevenue,TotalRevenue,BookingStatus,AgentID
414,452,None,None,Europa,None,Sydney Stellar Port,None,None,None,Cancelled,3


Following the status change from 'Pending' to 'Cancelled' for `AgentID` 3, the `load` value in `space_travel_agents` was successfully decremented, reflecting the agent’s completion of their customer assignment. As a result, `AgentID` 3 reclaimed the top rank in the system’s dynamic `agent_rank tracker`. This change demonstrated that both the trigger logic and ranking recalculation mechanisms were functioning as expected, ensuring agents are evaluated and prioritized in real time as workload conditions evolve.

Let's check the load

In [27]:
%sql SELECT * FROM space_travel_agents WHERE AgentID = 3

Running query in 'sqlite:///my_database.db'

AgentID,FirstName,LastName,Email,JobTitle,DepartmentName,ManagerName,SpaceLicenseNumber,YearsOfService,AverageCustomerServiceRating,load,agent_rank
3,Nova,Singh,nova.singh@astra2081.com,Senior Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-003,15,4.0,0,2


In [28]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
3,1
27,1
24,3
1,4
16,4
